In [1]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import pulp

In [3]:
df = pd.read_csv("solution scheme.txt")
df

,CONTAINER ROUTE
- X [ Transport -- source ; (0 : 0 on mode -1 ) TO Claveau ; (7 : 30 on mode 0 ),of container cont Q003 > ] = 1
- X [ Transport -- source ; (0 : 0 on mode -1 ) TO Parc-des-Expositions ; (7 : 29 on mode 1 ),of container cont Q002 > ] = 1
- X [ Transport -- source ; (0 : 0 on mode -1 ) TO Pin-Galant ; (7 : 12 on mode 2 ),of container cont Q001 > ] = 1
- X [ Transport -- Claveau ; (7 : 30 on mode 0 ) TO Quinconces ; (7 : 46 on mode 0 ),of container cont Q003 > ] = 1
- X [ indirect transshipment -- Quinconces ; (7 : 46 on mode 0 ) TO Quinconces ; (7 : 47 on mode -1 ),of container cont Q003 > ] = 1
- X [ waiting -- Quinconces ; (7 : 47 on mode -1 ) TO Quinconces ; (7 : 50 on mode 1 ),of container cont Q003 > ] = 1
- X [ Transport -- Parc-des-Expositions ; (7 : 29 on mode 1 ) TO Quinconces ; (7 : 50 on mode 1 ),of container cont Q002 > ] = 1
- X [ Transport -- Quinconces ; (7 : 50 on mode 1 ) TO Porte-de-Bourgogne ; (7 : 55 on mode 1 ),of container cont Q002 > ] = 1
- X [ Transport -- Quinconces ; (7 : 50 on mode 1 ) TO Porte-de-Bourgogne ; (7 : 55 on mode 1 ),of container cont Q003 > ] = 1
- X [ Transport -- Porte-de-Bourgogne ; (7 : 55 on mode 1 ) TO Gare-Saint-Jean ; (8 : 1 on mode 1 ),of container cont Q001 > ] = 1


In [4]:
df1 = pd.read_excel("parcel deliveries.xls")
df1

,PARCEL,CONTAINER,ORIGIN,DESTINATION,TIME,TIME2
0,P001,Q003,Claveau,Porte-de-Bourgogne,10.23,08:35:00
1,P002,Q003,Claveau,Gare-Saint-Jean,1.45,08:55:00
2,P003,Q003,Quinconces,Gare-Saint-Jean,1.45,08:55:00
3,P004,Q001,Hotel-de-Ville,Gare-de-Begles,1.45,08:55:00
4,P005,Q001,Hotel-de-Ville,Gare-Saint-Jean,1.45,08:55:00
5,P006,Q002,Parc-des-Expositions,Gare-Saint-Jean,1.45,08:55:00
6,P007,Q001,Pin-Galant,Gare-Saint-Jean,1.45,08:55:00


In [54]:
df2 = pd.read_excel("CONTAINER ROUTE DATA.xlsx")
df2

,STATUS,START_PATH,START_TIME,START_MODE,END_PATH,END_TIME,END_MODE,CONTAINER
0,Transport,source,00:00:00,-1,Claveau,07:30:00,0,Q003
1,Transport,source,00:00:00,-1,Parc-des-Expositions,07:29:00,1,Q002
2,Transport,source,00:00:00,-1,Pin-Galant,07:12:00,2,Q001
3,Transport,Pin-Galant,07:12:00,2,Fontaine-Arlac,07:24:00,2,Q001
4,Transport,Fontaine-Arlac,07:24:00,2,Hotel-de-Ville,07:42:00,2,Q001
5,Transport,Parc-des-Expositions,07:29:00,1,Quinconces,07:50:00,1,Q002
6,Transport,Claveau,07:30:00,0,Quinconces,07:46:00,0,Q003
7,Transport,Hotel-de-Ville,07:42:00,2,Porte-de-Bourgogne,07:46:00,2,Q001
8,indirect transshipment,Quinconces,07:46:00,0,Quinconces,07:47:00,-1,Q003
9,indirect transshipment,Porte-de-Bourgogne,07:46:00,2,Porte-de-Bourgogne,07:47:00,-1,Q001


In [45]:

# create a dictionary to map container ids to their contents
container_dict = {row['CONTAINER']: row['PARCEL'] for _, row in df1.iterrows()}

# create a list to store the parcel route
parcel_route = []

# loop through the rows of df2 and add each leg of the journey to the parcel route
for _, row in df2.iterrows():
    if row['STATUS'] == 'Transport':
        # extract the relevant information from the row
        start_path = row['START_PATH']
        start_time = row['START_TIME'].strftime('%H:%M')  # convert to string format
        start_mode = row['START_MODE']
        end_path = row['END_PATH']
        end_time = row['END_PATH2'].strftime('%H:%M')  # convert to string format
        end_mode = row['END_MODE']
        container = row['CONTAINER']
        parcel = container_dict[container]
        # create a string representation of the leg of the journey and add it to the parcel route
        leg_str = f"[ Transport -- {start_path} ; ({start_time}) ON mode {start_mode} ) TO {end_path} ; ({end_time}) , {parcel} in container cont {container} > ]"
        parcel_route.append(leg_str)
    elif row['STATUS'] == 'indirect transshipment':
        # extract the relevant information from the row
        start_path = row['START_PATH']
        start_time = row['START_TIME'].strftime('%H:%M')  # convert to string format
        start_mode = row['START_MODE']
        end_path = row['END_PATH']
        end_time = row['END_PATH2'].strftime('%H:%M')  # convert to string format
        end_mode = row['END_MODE']
        container = row['CONTAINER']
        parcel = container_dict[container]
        # create a string representation of the leg of the journey and add it to the parcel route
        leg_str = f"[ indirect transshipment -- {start_path} ; ({start_time}) TO {end_path} ; ({end_time}) ,{parcel} in container cont {container} > ]"
        parcel_route.append(leg_str)
    elif row['STATUS'] == 'waiting':
        # extract the relevant information from the row
        start_path = row['START_PATH']
        start_time = row['START_TIME'].strftime('%H:%M')  # convert to string format
        start_mode = row['START_MODE']
        end_path = row['END_PATH']
        end_time = row['END_PATH2'].strftime('%H:%M')  # convert to string format
        end_mode = row['END_MODE']
        container = row['CONTAINER']
        parcel = container_dict[container]
        # create a string representation of the leg of the journey and add it to the parcel route
        leg_str = f"[ waiting -- {start_path} ; ({start_time}) TO {end_path} ; ({end_time}) , {parcel} in container cont {container} > ]"
        parcel_route.append(leg_str)

# print the final destination of the last leg of
print(f"FINAL DESTINATION {container_dict[df2.iloc[-1]['CONTAINER']]}")

# print the parcel route
for leg in parcel_route:
    print(leg)


FINAL DESTINATION P003
[ Transport -- source ; (00:00) ON mode -1 ) TO Claveau ; (07:30) , P003 in container cont Q003 > ]
[ Transport -- source ; (00:00) ON mode -1 ) TO Parc-des-Expositions ; (07:29) , P006 in container cont Q002 > ]
[ Transport -- source ; (00:00) ON mode -1 ) TO Pin-Galant ; (07:12) , P007 in container cont Q001 > ]
[ Transport -- Pin-Galant ; (07:12) ON mode 2 ) TO Fontaine-Arlac ; (07:24) , P007 in container cont Q001 > ]
[ Transport -- Fontaine-Arlac ; (07:24) ON mode 2 ) TO Hotel-de-Ville ; (07:42) , P007 in container cont Q001 > ]
[ Transport -- Parc-des-Expositions ; (07:29) ON mode 1 ) TO Quinconces ; (07:50) , P006 in container cont Q002 > ]
[ Transport -- Claveau ; (07:30) ON mode 0 ) TO Quinconces ; (07:46) , P003 in container cont Q003 > ]
[ Transport -- Hotel-de-Ville ; (07:42) ON mode 2 ) TO Porte-de-Bourgogne ; (07:46) , P007 in container cont Q001 > ]
[ indirect transshipment -- Quinconces ; (07:46) TO Quinconces ; (07:47) ,P003 in container cont Q00

In [57]:
# create a dictionary to map container ids to their contents
container_dict = {row['CONTAINER']: row['PARCEL'] for _, row in df1.iterrows()}

# create a list to store the parcel route
parcel_route = []

# loop through the rows of df2 and add each leg of the journey to the parcel route
for _, row in df2.iterrows():
    if row['STATUS'] == 'Transport':
        # extract the relevant information from the row
        start_path = row['START_PATH']
        start_time = row['START_TIME']
        start_mode = row['START_MODE']
        end_path = row['END_PATH']
        end_time = row['END_TIME']
        end_mode = row['END_MODE']
        container = row['CONTAINER']
        parcel = container_dict[container]
        # create a dictionary representation of the leg of the journey and add it to the parcel route
        leg_dict = {
            'status': 'Transport',
            'start_path': start_path,
            'start_time': start_time,
            'start_mode': start_mode,
            'end_path': end_path,
            'end_time': end_time,
            'end_mode': end_mode,
            'parcel': parcel,
            'container': container
        }
        parcel_route.append(leg_dict)
    elif row['STATUS'] == 'indirect transshipment':
        # extract the relevant information from the row
        start_path = row['START_PATH']
        start_time = row['START_TIME']
        start_mode = row['START_MODE']
        end_path = row['END_PATH']
        end_time = row['END_TIME']
        end_mode = row['END_MODE']
        container = row['CONTAINER']
        parcel = container_dict[container]
        # create a dictionary representation of the leg of the journey and add it to the parcel route
        leg_dict = {
            'status': 'indirect transshipment',
            'start_path': start_path,
            'start_time': start_time,
            'start_mode': start_mode,
            'end_path': end_path,
            'end_time': end_time,
            'end_mode': end_mode,
            'parcel': parcel,
            'container': container
        }
        parcel_route.append(leg_dict)
    elif row['STATUS'] == 'waiting':
        # extract the relevant information from the row
        start_path = row['START_PATH']
        start_time = row['START_TIME']
        start_mode = row['START_MODE']
        end_path = row['END_PATH']
        end_time = row['END_TIME']
        end_mode = row['END_MODE']
        container = row['CONTAINER']
        parcel = container_dict[container]
        # create a dictionary representation of the leg of the journey and add it to the parcel route
        leg_dict = {
            'status': 'waiting',
            'start_path': start_path,
            'start_time': start_time,
            'start_mode': start_mode,
            'end_path': end_path,
            'end_time': end_time,
            'end_mode': end_mode,
            'parcel': parcel,
            'container': container
        }
        parcel_route.append(leg_dict)

# group the list of leg dictionaries by parcel
grouped_parcel_route = pd.DataFrame(parcel_route)
print(grouped_parcel_route)

                    status            start_path start_time  start_mode  \
0                Transport                source   00:00:00          -1   
1                Transport                source   00:00:00          -1   
2                Transport                source   00:00:00          -1   
3                Transport            Pin-Galant   07:12:00           2   
4                Transport        Fontaine-Arlac   07:24:00           2   
5                Transport  Parc-des-Expositions   07:29:00           1   
6                Transport               Claveau   07:30:00           0   
7                Transport        Hotel-de-Ville   07:42:00           2   
8   indirect transshipment            Quinconces   07:46:00           0   
9   indirect transshipment    Porte-de-Bourgogne   07:46:00           2   
10                 waiting            Quinconces   07:47:00          -1   
11                 waiting    Porte-de-Bourgogne   07:47:00          -1   
12               Transpor

In [58]:
# create a dictionary of paths
path_dict = {}
for index, row in df2.iterrows():
    path_dict[row['CONTAINER']] = path_dict.get(row['CONTAINER'], []) + [(row['START_PATH'], row['START_TIME'], row['START_MODE']), (row['END_PATH'], row['END_TIME'], row['END_MODE'])]

# create parcel routes
parcel_routes = []
for index, row in df1.iterrows():
    parcel = row['PARCEL']
    container = row['CONTAINER']
    origin = row['ORIGIN']
    destination = row['DESTINATION']
    route = []

    # check if container exists in path dictionary
    if container in path_dict:
        path = path_dict[container]

        # add starting transport to route
        if len(path) > 0:
            start = path[0]
            if start[0] == origin:
                route.append(f"[ Transport -- source ; (0 : 0 on mode -1 ) TO {origin} ; ({start[1]} on mode {start[2]}) ,  {parcel} in container cont {container} > ]")
                path.pop(0)

        # add all intermediate transports to route
        for i in range(len(path)-1):
            current = path[i]
            next_ = path[i+1]
            if current[0] == destination:
                break
            if current[0] != next_[0]:
                route.append(f"[ Transport -- {current[0]} ; ({current[1]} on mode {current[2]}) TO {next_[0]} ; ({next_[1]} on mode {next_[2]}) ,  {parcel} in container cont {container} > ]")

        # add final transport to destination
        if len(path) > 0:
            end = path[-1]
            if end[0] == destination:
                route.append(f"[ Transport -- {end[0]} ; ({end[1]} on mode {end[2]}) TO sink ; (23 : 59 on mode -1 ) ,  {parcel} in container cont {container} > ]")

    parcel_routes.append(route)

# print parcel routes
for route in parcel_routes:
    print(route)


['[ Transport -- source ; (00:00:00 on mode -1) TO Claveau ; (07:30:00 on mode 0) ,  P001 in container cont Q003 > ]', '[ Transport -- Claveau ; (07:30:00 on mode 0) TO Quinconces ; (07:46:00 on mode 0) ,  P001 in container cont Q003 > ]', '[ Transport -- Quinconces ; (07:50:00 on mode 1) TO Porte-de-Bourgogne ; (07:55:00 on mode 1) ,  P001 in container cont Q003 > ]']
['[ Transport -- source ; (00:00:00 on mode -1) TO Claveau ; (07:30:00 on mode 0) ,  P002 in container cont Q003 > ]', '[ Transport -- Claveau ; (07:30:00 on mode 0) TO Quinconces ; (07:46:00 on mode 0) ,  P002 in container cont Q003 > ]', '[ Transport -- Quinconces ; (07:50:00 on mode 1) TO Porte-de-Bourgogne ; (07:55:00 on mode 1) ,  P002 in container cont Q003 > ]', '[ Transport -- Porte-de-Bourgogne ; (07:55:00 on mode 1) TO Gare-Saint-Jean ; (08:01:00 on mode 1) ,  P002 in container cont Q003 > ]']
['[ Transport -- source ; (00:00:00 on mode -1) TO Claveau ; (07:30:00 on mode 0) ,  P003 in container cont Q003 > ]', 

In [62]:
# create a dictionary of paths
path_dict = {}
for index, row in df2.iterrows():
    path_dict[row['CONTAINER']] = path_dict.get(row['CONTAINER'], []) + [(row['START_PATH'], row['START_TIME'], row['START_MODE']), (row['END_PATH'], row['END_TIME'], row['END_MODE'])]

# create parcel routes for each container separately
container_routes = {}
for index, row in df1.iterrows():
    parcel = row['PARCEL']
    container = row['CONTAINER']
    origin = row['ORIGIN']
    destination = row['DESTINATION']
    route = []

    # check if container exists in path dictionary
    if container in path_dict:
        path = path_dict[container]

        # add starting transport to route
        if len(path) > 0:
            start = path[0]
            if start[0] == origin:
                route.append(f"[ Transport -- source ; (0 : 0 on mode -1 ) TO {origin} ; ({start[1]} on mode {start[2]}) ,  {parcel} in container cont {container} > ]")
                path.pop(0)

        # add all intermediate transports to route
        for i in range(len(path)-1):
            current = path[i]
            next_ = path[i+1]
            if current[0] == destination:
                break
            if current[0] != next_[0]:
                route.append(f"[ Transport -- {current[0]} ; ({current[1]} on mode {current[2]}) TO {next_[0]} ; ({next_[1]} on mode {next_[2]}) ,  {parcel} in container cont {container} > ]")

        # add final transport to destination
        if len(path) > 0:
            end = path[-1]
            if end[0] == destination:
                route.append(f"[ Transport -- {end[0]} ; ({end[1]} on mode {end[2]}) TO sink ; (23 : 59 on mode -1 ) ,  {parcel} in container cont {container} > ]")

    if container in container_routes:
        container_routes[container].append(route)
    else:
        container_routes[container] = [route]

# print parcel routes for each container separately
for container, routes in container_routes.items():
    print(f"Container {container}:")
    for route in routes:
        print(route)


Container Q003:
['[ Transport -- source ; (00:00:00 on mode -1) TO Claveau ; (07:30:00 on mode 0) ,  P001 in container cont Q003 > ]', '[ Transport -- Claveau ; (07:30:00 on mode 0) TO Quinconces ; (07:46:00 on mode 0) ,  P001 in container cont Q003 > ]', '[ Transport -- Quinconces ; (07:50:00 on mode 1) TO Porte-de-Bourgogne ; (07:55:00 on mode 1) ,  P001 in container cont Q003 > ]']
['[ Transport -- source ; (00:00:00 on mode -1) TO Claveau ; (07:30:00 on mode 0) ,  P002 in container cont Q003 > ]', '[ Transport -- Claveau ; (07:30:00 on mode 0) TO Quinconces ; (07:46:00 on mode 0) ,  P002 in container cont Q003 > ]', '[ Transport -- Quinconces ; (07:50:00 on mode 1) TO Porte-de-Bourgogne ; (07:55:00 on mode 1) ,  P002 in container cont Q003 > ]', '[ Transport -- Porte-de-Bourgogne ; (07:55:00 on mode 1) TO Gare-Saint-Jean ; (08:01:00 on mode 1) ,  P002 in container cont Q003 > ]']
['[ Transport -- source ; (00:00:00 on mode -1) TO Claveau ; (07:30:00 on mode 0) ,  P003 in container 

In [145]:
# create a dictionary of paths
path_dict = {}
for index, row in df2.iterrows():
    path_dict[row['CONTAINER']] = path_dict.get(row['CONTAINER'], []) + [(row['START_PATH'], row['START_TIME'], row['START_MODE']), (row['END_PATH'], row['END_TIME'], row['END_MODE'])]

# create parcel routes
parcel_routes = []
for index, row in df1.iterrows():
    parcel = row['PARCEL']
    container = row['CONTAINER']
    origin = row['ORIGIN']
    destination = row['DESTINATION']
    route = []

    # check if container exists in path dictionary
    if container in path_dict:
        path = path_dict[container]

        # add starting transport to route
        if len(path) > 0:
            start = path[0]
            if start[0] == origin:
                route.append(f"[ Transport -- source ; (0 : 0 on mode -1 ) TO {origin} ; ({start[1]} on mode {start[2]}) ,  {parcel} in container cont {container} > ]")
                path.pop(0)

        # add all intermediate transports to route
        for i in range(len(path)-1):
            current = path[i]
            next_ = path[i+1]
            if current[0] == destination:
                break
            if current[0] != next_[0]:
                route.append(f"[ Transport -- {current[0]} ; ({current[1]} on mode {current[2]}) TO {next_[0]} ; ({next_[1]} on mode {next_[2]}) ,  {parcel} in container cont {container} > ]")

        # add final transport to destination
        if len(path) > 0:
            end = path[-1]
            if end[0] == destination:
                route.append(f"[ Transport -- {end[0]} ; ({end[1]} on mode {end[2]}) TO sink ; (23 : 59 on mode -1 ) ,  {parcel} in container cont {container} > ]")

    parcel_routes.append(route)

# create a dataframe with parcel routes
df_routes1 = pd.DataFrame({'Parcel': df1['PARCEL'], 'Container': df1['CONTAINER'], 'Origin':df1['ORIGIN'], 'Destination':df1['DESTINATION'], 'Routes': parcel_routes})

# export dataframe to Excel file
df_routes1.to_excel('parcel_routes.xlsx', index=False)


In [146]:
df_routes1

,Parcel,Container,Origin,Destination,Routes
0,P001,Q003,Claveau,Porte-de-Bourgogne,[[ Transport -- source ; (00:00:00 on mode -1)...
1,P002,Q003,Claveau,Gare-Saint-Jean,[[ Transport -- source ; (00:00:00 on mode -1)...
2,P003,Q003,Quinconces,Gare-Saint-Jean,[[ Transport -- source ; (00:00:00 on mode -1)...
3,P004,Q001,Hotel-de-Ville,Gare-de-Begles,[[ Transport -- source ; (00:00:00 on mode -1)...
4,P005,Q001,Hotel-de-Ville,Gare-Saint-Jean,[[ Transport -- source ; (00:00:00 on mode -1)...
5,P006,Q002,Parc-des-Expositions,Gare-Saint-Jean,[[ Transport -- source ; (00:00:00 on mode -1)...
6,P007,Q001,Pin-Galant,Gare-Saint-Jean,[[ Transport -- source ; (00:00:00 on mode -1)...


In [115]:
df_routes1.head(1)

,Parcel,Container,Routes
0,P001,Q003,[[ Transport -- source ; (00:00:00 on mode -1)...


In [116]:
print(df_routes1.loc[0])

Parcel                                                    P001
Container                                                 Q003
Routes       [[ Transport -- source ; (00:00:00 on mode -1)...
Name: 0, dtype: object


In [117]:
print(df_routes1.loc[0, [ 'Routes']])

Routes    [[ Transport -- source ; (00:00:00 on mode -1)...
Name: 0, dtype: object


In [160]:
# create a dictionary of paths
path_dict = {}
for index, row in df2.iterrows():
    path_dict[row['CONTAINER']] = path_dict.get(row['CONTAINER'], []) + [(row['START_PATH'], row['START_TIME'], row['START_MODE']), (row['END_PATH'], row['END_TIME'], row['END_MODE'])]

# create transport routes
transport_routes = []
for index, row in df1.iterrows():
    parcel = row['PARCEL']
    container = row['CONTAINER']
    origin = row['ORIGIN']
    destination = row['DESTINATION']

    # check if container exists in path dictionary
    if container in path_dict:
        path = path_dict[container]

        # add starting transport to route
        if len(path) > 0:
            start = path[0]
            if start[0] == origin:
                transport_routes.append([parcel, container, 'source', '0', '0', '-1', origin, start[1], start[2]])

        # add all intermediate transports to route
        for i in range(len(path)-1):
            current = path[i]
            next_ = path[i+1]
            if current[0] == destination:
                break
            if current[0] != next_[0]:
                transport_routes.append([parcel, container, current[0], current[1], current[2], next_[2], next_[0], next_[1], next_[2]])

        # add final transport to destination
        if len(path) > 0:
            end = path[-1]
            if end[0] == destination:
                transport_routes.append([parcel, container, end[0], end[1], end[2], '-1', 'sink', '23', '-1'])

# create a dataframe with transport routes
df_routes2 = pd.DataFrame(transport_routes, columns=['Parcel', 'Container', 'Origin', 'Origin_Time', 'Origin_Mode', 'Destination_Mode', 'Destination', 'Destination_Time', 'Destination_Mode'])

# export dataframe to Excel file
df_routes2.to_excel('transport_routes.xlsx', index=False)


In [148]:
df_routes2

,Parcel,Container,Origin,Origin_Time,Origin_Mode,Destination_Mode,Destination,Destination_Time,Destination_Mode
0,P001,Q003,source,00:00:00,-1,0,Claveau,07:30:00,0
1,P001,Q003,Claveau,07:30:00,0,0,Quinconces,07:46:00,0
2,P001,Q003,Quinconces,07:50:00,1,1,Porte-de-Bourgogne,07:55:00,1
3,P002,Q003,source,00:00:00,-1,0,Claveau,07:30:00,0
4,P002,Q003,Claveau,07:30:00,0,0,Quinconces,07:46:00,0
5,P002,Q003,Quinconces,07:50:00,1,1,Porte-de-Bourgogne,07:55:00,1
6,P002,Q003,Porte-de-Bourgogne,07:55:00,1,1,Gare-Saint-Jean,08:01:00,1
7,P003,Q003,source,00:00:00,-1,0,Claveau,07:30:00,0
8,P003,Q003,Claveau,07:30:00,0,0,Quinconces,07:46:00,0
9,P003,Q003,Quinconces,07:50:00,1,1,Porte-de-Bourgogne,07:55:00,1


In [149]:
P001 = df_routes2[0:3]
P001

,Parcel,Container,Origin,Origin_Time,Origin_Mode,Destination_Mode,Destination,Destination_Time,Destination_Mode
0,P001,Q003,source,00:00:00,-1,0,Claveau,07:30:00,0
1,P001,Q003,Claveau,07:30:00,0,0,Quinconces,07:46:00,0
2,P001,Q003,Quinconces,07:50:00,1,1,Porte-de-Bourgogne,07:55:00,1


In [150]:
P002 = df_routes2[3:7]
P002

,Parcel,Container,Origin,Origin_Time,Origin_Mode,Destination_Mode,Destination,Destination_Time,Destination_Mode
3,P002,Q003,source,00:00:00,-1,0,Claveau,07:30:00,0
4,P002,Q003,Claveau,07:30:00,0,0,Quinconces,07:46:00,0
5,P002,Q003,Quinconces,07:50:00,1,1,Porte-de-Bourgogne,07:55:00,1
6,P002,Q003,Porte-de-Bourgogne,07:55:00,1,1,Gare-Saint-Jean,08:01:00,1


In [151]:
P003 = df_routes2[7:11]
P003

,Parcel,Container,Origin,Origin_Time,Origin_Mode,Destination_Mode,Destination,Destination_Time,Destination_Mode
7,P003,Q003,source,00:00:00,-1,0,Claveau,07:30:00,0
8,P003,Q003,Claveau,07:30:00,0,0,Quinconces,07:46:00,0
9,P003,Q003,Quinconces,07:50:00,1,1,Porte-de-Bourgogne,07:55:00,1
10,P003,Q003,Porte-de-Bourgogne,07:55:00,1,1,Gare-Saint-Jean,08:01:00,1


In [152]:
P004 = df_routes2[14:17]
P004

,Parcel,Container,Origin,Origin_Time,Origin_Mode,Destination_Mode,Destination,Destination_Time,Destination_Mode
14,P004,Q001,Hotel-de-Ville,07:42:00,2,2,Porte-de-Bourgogne,07:46:00,2
15,P004,Q001,Porte-de-Bourgogne,07:55:00,1,1,Gare-Saint-Jean,08:01:00,1
16,P004,Q001,Gare-Saint-Jean,08:01:00,1,1,Gare-de-Begles,08:13:00,1


In [154]:
P005 = df_routes2[20:22]
P005

,Parcel,Container,Origin,Origin_Time,Origin_Mode,Destination_Mode,Destination,Destination_Time,Destination_Mode
20,P005,Q001,Hotel-de-Ville,07:42:00,2,2,Porte-de-Bourgogne,07:46:00,2
21,P005,Q001,Porte-de-Bourgogne,07:55:00,1,1,Gare-Saint-Jean,08:01:00,1


In [155]:
P006 = df_routes2[22:26]
P006

,Parcel,Container,Origin,Origin_Time,Origin_Mode,Destination_Mode,Destination,Destination_Time,Destination_Mode
22,P006,Q002,source,00:00:00,-1,1,Parc-des-Expositions,07:29:00,1
23,P006,Q002,Parc-des-Expositions,07:29:00,1,1,Quinconces,07:50:00,1
24,P006,Q002,Quinconces,07:50:00,1,1,Porte-de-Bourgogne,07:55:00,1
25,P006,Q002,Porte-de-Bourgogne,07:55:00,1,1,Gare-Saint-Jean,08:01:00,1


In [156]:
P007 = df_routes2[26:31]
P007

,Parcel,Container,Origin,Origin_Time,Origin_Mode,Destination_Mode,Destination,Destination_Time,Destination_Mode
26,P007,Q001,source,00:00:00,-1,2,Pin-Galant,07:12:00,2
27,P007,Q001,Pin-Galant,07:12:00,2,2,Fontaine-Arlac,07:24:00,2
28,P007,Q001,Fontaine-Arlac,07:24:00,2,2,Hotel-de-Ville,07:42:00,2
29,P007,Q001,Hotel-de-Ville,07:42:00,2,2,Porte-de-Bourgogne,07:46:00,2
30,P007,Q001,Porte-de-Bourgogne,07:55:00,1,1,Gare-Saint-Jean,08:01:00,1


In [142]:
# create a dictionary of paths
path_dict = {}
for index, row in df2.iterrows():
    path_dict[row['CONTAINER']] = path_dict.get(row['CONTAINER'], []) + [(row['START_PATH'], row['START_TIME'], row['START_MODE']), (row['END_PATH'], row['END_TIME'], row['END_MODE'])]

# create parcel routes
parcel_routes = []
for index, row in df1.iterrows():
    parcel = row['PARCEL']
    container = row['CONTAINER']
    origin = row['ORIGIN']
    destination = row['DESTINATION']
    route = []

    # check if container exists in path dictionary
    if container in path_dict:
        path = path_dict[container]

        # add starting transport to route
        if len(path) > 0:
            start = path[0]
            if start[0] == origin:
                route.append(f"[ Transport -- source ; (0 : 0 on mode -1 ) TO {origin} ; ({start[1]} on mode {start[2]}) ,  {parcel} in container cont {container} > ]")
                path.pop(0)

        # add all intermediate transports to route
        for i in range(len(path)-1):
            current = path[i]
            next_ = path[i+1]
            if current[0] == destination:
                break
            if current[0] != next_[0]:
                route.append(f"[ Transport -- {current[0]} ; ({current[1]} on mode {current[2]}) TO {next_[0]} ; ({next_[1]} on mode {next_[2]}) ,  {parcel} in container cont {container} > ]")
            else:
                route.append(f"[ indirect transshipment -- {current[0]} ; ({current[1]} on mode {current[2]}) TO {next_[0]} ; ({next_[1]} on mode {next_[2]}) ,{parcel} in container cont {container} > ]")
                route.append(f"[ waiting -- {next_[0]} ; ({next_[1]} on mode -1 ) TO {next_[0]} ; ({next_[1]} on mode {next_[2]}) ,  {parcel} in container cont {container} > ]")

        # add final transport to destination
        if len(path) > 0:
            end = path[-1]
            if end[0] == destination:
                route.append(f"[ Transport -- {end[0]} ; ({end[1]} on mode {end[2]}) TO sink ; (23 : 59 on mode -1 ) ,  {parcel} in container cont {container} > ]")
            else:
                route.append(f"[ Loading -- {end[0]} ; ({end[1]} on mode {end[2]}) TO {destination} ; (23 : 59 on mode -1 ) ,  {parcel} in container cont {container} > ]")
                route.append(f"[ Transport -- {destination} ; (0 : 0 on mode -1 ) TO sink ; (23 : 59 on mode -1 ) ,  {parcel} in container cont {container} > ]")

    parcel_routes.append(route)

# create a dataframe with parcel routes
df_routes3 = pd.DataFrame({'Parcel': df1['PARCEL'], 'Container': df1['CONTAINER'], 'Routes': parcel_routes})

# export dataframe to Excel file
df_routes3.to_excel('parcel_routes1.xlsx', index=False)
